In [ ]:
import requests
import os
import transitfeed
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import datetime

API_KEY = "7e298dee-869b-4522-af6e-22b5ee9395d0"

# Download GTFS feeds

In [259]:
feeds = requests.get("https://api.transitfeeds.com/v1/getFeeds?key=" + API_KEY + "&location=67&descendants=1&page=1&limit=100&type=gtfs").json()
results = feeds['results']['feeds']

feeds = requests.get("https://api.transitfeeds.com/v1/getFeeds?key=" + API_KEY + "&location=67&descendants=1&page=2&limit=100&type=gtfs").json()
results.extend(feeds['results']['feeds'])

missing = [f for f in feeds['results']['feeds'] if 'd' not in f['u']]

urls = [(f['t'], f['u']['d']) for f in feeds['results']['feeds'] if 'd' in f['u']]

for (name, url) in urls:
    os.mkdir('GTFS/' + name)
    r = requests.get(url, allow_redirects=True)
    open('GTFS/' + name + '/google_transit.zip', 'wb').write(r.content)

# Process GTFS feeds

In [ ]:
folders = [f for f in os.listdir('./GTFS/') if not os.path.isfile(f)]
for folder in folders:
    if not os.path.exists('./Analyzed/' + folder + '.geojson'):
        files = os.listdir('./GTFS/'+folder+'/')
        for f in files:
            if f[-3:] == 'zip':
                fname = './GTFS/'+folder+'/'+f
                try:
                    fetch_stops(fname, folder)
                except:
                    print "ERROR: " + fname
    else:
        print folder + " already exists!"

In [7]:
def fetch_stops(fname, folder):
    print(fname)
    loader = transitfeed.Loader(fname)
    schedule = loader.Load()

    stop_data = []
    t = len(schedule.stops.values())
    i = 0
    print str(t) + " stops"
    date_bound = datetime.datetime.now()
    
    if datetime.datetime.strptime(schedule.GetDateRange()[1], '%Y%m%d') < date_bound:
        print "Schedule ends on " + schedule.GetDateRange()[1] + ". This might mean an old GTFS file"
        date_bound = datetime.datetime.strptime(schedule.GetDateRange()[1], '%Y%m%d')
        

    for stop in schedule.stops.values():
        if i % 500 == 0:
            print i
            
        i += 1

        info = get_stop_data(stop, schedule, folder, date_bound)
        stop_data.append(info)
        
    geojson = make_geojson(stop_data)        
    json.dump(geojson, open('./Analyzed/' + folder + '.geojson', 'w'))

def make_geojson(info):
    geojson = {
      "type": "FeatureCollection",
      "features": []
    }
    
    for i in info:
        feature = {
            "type": "Feature",
            "properties": i,
            "geometry": {
                "type": "Point",
                "coordinates": [i["lon"], i["lat"]]
            }
        }
        
        geojson["features"].append(feature)
        
    return geojson    

def get_stop_data(stop, schedule, folder, date_bound):
    times = []
    trainOrFerry = False
    for stop_time in stop.GetStopTimeTrips(schedule):
        trip = stop_time[1][0]
        route = schedule.GetRoute(trip.route_id)
        trip_schedule = schedule.GetServicePeriod(trip.service_id)
        if trip_schedule.start_date:
            start_date = trip_schedule.start_date
        else:
            start_date = trip_schedule.GetDateRange()[0]
            
        if trip_schedule.end_date:
            end_date = trip_schedule.end_date
        else:
            end_date = trip_schedule.GetDateRange()[1]
        
        if datetime.datetime.strptime(end_date, '%Y%m%d') >= date_bound and datetime.datetime.strptime(start_date, '%Y%m%d') < date_bound:
            if (route.route_type == 0 or route.route_type == 1 or route.route_type == 2 or route.route_type == 4):
                trainOrFerry = True
                
            if not any(trip_schedule.day_of_week):
                # not valid for any days of the week
                days = trip_schedule.ActiveDates()
                weekday = '20190506' in days and '20190507' in days and '20190508' in days and '20190509' in days and '20190510' in days
                saturday = '20190504' in days
                sunday = '20190505' in days
            else:
                weekday =  all(trip_schedule.day_of_week[:5])
                saturday = trip_schedule.day_of_week[5]
                sunday = trip_schedule.day_of_week[6]

            times.append((route.route_short_name,
                trip.direction_id,
                stop_time[0],
                weekday,
                saturday,
                sunday))

    df = pd.DataFrame(times, columns=['Route', 'Direction', 'Time', 'M-F', 'Sa', 'Su']).set_index('Time')
    grp = df.groupby(('Route', 'Direction', 'M-F', 'Sa', 'Su'))

    bestAmPeak = 9999
    bestAmSchedule = []
    amRoute = ''
    
    bestPmPeak = 9999
    bestPmSchedule = []
    pmRoute = ''
    
    alldayMF = 9999
    bestAlldaySchedule = []
    wkRoute = ''
    
    bestAlldaySaSchedule = []
    alldaySa = 9999
    saRoute = ''
    
    bestAlldaySuSchedule = []
    alldaySu = 9999
    suRoute = ''

    for group in grp.groups:
        index = grp.get_group(group).index.sort_values()
        if (len(index) >= 2):
#             print index
#             print list(index)
#             print set(list(index))
            index = list(set(list(index)))
            index.sort()

            if (group[2]):
                (amPeak, amSchedule) = get_peak_average_interval(index, 6, 10)
                (pmPeak, pmSchedule) = get_peak_average_interval(index, 15, 19)

                if amPeak < bestAmPeak:
                    bestAmPeak = amPeak
                    bestAmSchedule = amSchedule
                    amRoute = group[0]

                if pmPeak < bestPmPeak:
                    bestPmPeak = pmPeak
                    bestPmSchedule = pmSchedule
                    pmRoute = group[0]

                (allday, alldaySchedule) = get_average_interval(index, 6, 22)

                if (allday < alldayMF):
                    alldayMF = allday
                    bestAlldaySchedule = alldaySchedule
                    wkRoute = group[0]

            if (group[3]):
                (allday, alldaySchedule) = get_average_interval(index, 8, 22)
                if allday < alldaySa:
                    alldaySa = allday
                    bestAlldaySaSchedule = alldaySchedule  
                    saRoute = group[0]

            if (group[4]):
                (allday, alldaySchedule) = get_average_interval(index, 8, 22)
                if allday < alldaySu:
                    alldaySu = allday
                    bestAlldaySuSchedule = alldaySchedule 
                    suRoute = group[0]

    alldayWknd = max(alldaySa, alldaySu)
    if bestPmPeak > bestAmPeak:
        bestSchedule = bestPmSchedule
    else:
        bestSchedule = bestAmSchedule
        
    if alldaySa > alldaySu:
        bestWeekendSchedule = bestAlldaySaSchedule
    else:
        bestWeekendSchedule = bestAlldaySuSchedule

    info = {'name': stop.stop_name,
            'lat': stop.stop_lat,
            'lon': stop.stop_lon,
            'amPeak': bestAmPeak,
            'amPeakSchedule': bestAmSchedule,
            'amRoute': amRoute,
            'pmPeak': bestPmPeak,
            'pmPeakSchedule': bestPmSchedule,
            'pmRoute': pmRoute,
            'peak': max(bestAmPeak, bestPmPeak),
            'peakSchedule': bestSchedule,     
            'weekday': alldayMF,
            'weekdaySchedule': bestAlldaySchedule,
            'weekdayRoute': wkRoute,
            'saturday': alldaySa,
            'saturdaySchedule': bestAlldaySaSchedule,
            'saturdayRoute': saRoute,
            'sunday': alldaySu,
            'sundaySchedule': bestAlldaySuSchedule,
            'sundayRoute': suRoute,
            'weekend': max(alldaySa, alldaySu),
            'weekendSchedule': bestWeekendSchedule,
            'trainOrFerry': trainOrFerry,
            'agency': folder}
    
    if (info['peak'] < 10 and info['weekday'] < 20 and info['weekend'] < 30) or info['trainOrFerry']:
        info['qualifies'] = True
    else:
        info['qualifies'] = False

    return info

def get_average_interval(index, start_h,end_h):
    index = list(index)
    if (len(index) < 2):
        return (9999, index)
    
    a = 0
    b = len(index)
    
    for i in range(len(index)):
        t = index[i]
        if t >= start_h*60*60:
            a = max(0, i - 1)
            break
    
    for i in range(len(index)):
        t = index[i]
        if t > end_h*60*60:
            b = min(len(index), i + 1)
            break
                        
    filtered = index[a:b]
    
    if (filtered[-1] - filtered[0] < 60*60*(end_h - start_h)):
        return (9998, filtered)

    intervals = [filtered[i+1] - filtered[i] for i in range(len(filtered)-1)]
    
    minvalue = np.mean(intervals)/60
    return (minvalue, filtered)

def get_peak_average_interval(index, start_h,end_h):
    filtered = [i for i in index if i >= start_h*60*60 and i <= end_h*60*60]

    if (len(index) < 2):
        return (9999, filtered)
    
    filtered = list(filtered)
    intervals = [filtered[i+1] - filtered[i] for i in range(len(filtered)-1)]
    
    (minvalue, bounds) = get_min_three_hour(intervals)
        
    return (minvalue/60, filtered[bounds[0]:bounds[1]+2])

def get_min_three_hour(intervals):
    mini = 9999*60

    a = 0
    b = len(intervals) - 1
    while b > a:

        s = intervals[a:b+1]
        if np.mean(s) < mini and np.sum(s) >= 3*60*60:
            mini = np.mean(s)

        nextl = s[0]
        nextr = s[-1]
        if nextl > nextr:
            if (np.sum(s) - nextl) >= 3*60*60:
                a += 1
            elif (np.sum(s) - nextr) >= 3*60*60:
                b -= 1
            else:
                return (mini, (a, b))
        else:
            if (np.sum(s) - nextr) >= 3*60*60:
                b -= 1
            elif (np.sum(s) - nextl) >= 3*60*60:
                a += 1
            else:
                return (mini, (a, b))
            
    return (mini, (a, b))